In [1]:
import copy

import numpy as np
#first some imports
import torch
torch.set_default_dtype(torch.float64)  # double precision for numerical stability

import matplotlib.pyplot as plt

import pyro
import pyro.distributions as dist
import pyro.poutine as poutine

In [2]:
def Infer(sampling_func, num_samples=50):
    """
    Estimate a distribution by importance sampling
    """
    return(pyro.infer.EmpiricalMarginal(
        pyro.infer.Importance(
            sampling_func, num_samples=num_samples).run()))

# Game env

Aux. functions from Saskia

In [3]:
def CardPlayable(card, fireworks):
    if card['color'] is None or card['rank'] is None:
        return False
    if fireworks[card['color']] == card['rank']:
        return True
    else:
        return False

# Card is not needed anymore in the future
def CardUseless(card, fireworks):
    if card['color'] is None or card['rank'] is None:
        return False
    if fireworks[card['color']] > int(card['rank']):
            return True
    else:
        return False

def remaining_copies(card, discard_pile):
    if card['rank'] == 1:
        total_copies = 3
    elif card['rank'] == 5:
        total_copies = 1
    else:
        total_copies = 2

    discarded_copies = discard_pile.count(str(card['color']) + str(card['rank']))

    return total_copies - discarded_copies

In [4]:
estimated_board = {'life_tokens': 3,
                   'information_tokens': 7,
                   'fireworks': {'R': 3, 'Y': 2, 'G': 4, 'W': 1, 'B': 2},
                   'discard_pile': ['B2', 'B3', 'B2'],
                   'estimated_hands': [
                       [{'color': None, 'rank': -1},
                        {'color': None, 'rank': 0},
                        {'color': None, 'rank': -1},
                        {'color': None, 'rank': -1},
                        {'color': None, 'rank': -1}],
                       [{'color': 'B', 'rank': 2},
                        {'color': 'R', 'rank': 0},
                        {'color': 'Y', 'rank': 4},
                        {'color': 'B', 'rank': 3},
                        {'color': 'Y', 'rank': 0}]
                    ]
                   }

last_action = {'action_type': 'REVEAL_COLOR', 'target_offset': 1, 'color': 'B'}

## Create all theoretical possible actions ##

In [60]:
# Bianca's Addition: Possible Actions

# from the Google Deepmind Hanabi Learning Environment:
COLOR_CHAR = ["R", "Y", "G", "W", "B"] 
ACTION_TYPES = ['PLAY', 'DISCARD', 'REVEAL_COLOR', 'REVEAL_RANK', 'DEAL', 'INVALID'] # ignore last two actions
# more global variables I added:
RANK = [0,1,2,3,4]
NUMBER_OF_PLAYERS = 2
POSITIONS = [1,2,3,4,5]

# creating the possible target_offset values for the game dependend on the number of players
# target_offset determines which player you give a hint to (1 = next in line, 2 = 2nd next and so on...)
target_offsets = []
for n in range(NUMBER_OF_PLAYERS):
    # ommit offset=0 as you can't give hints to yourself
    if n > 0:
        target_offsets.append(n)
        
# dictionary with all possible values for all possible actions 
possible_actions_lexicon = {'action_type': ACTION_TYPES,'target_offset': target_offsets,
                            'color': COLOR_CHAR, 'rank': RANK, 'card_index': POSITIONS}

# function for creating an instance of an action
def create_action_instance(act_type, info, target_offset, card_index):
    
    # is it a color or rank hint or no hint at all
    if act_type is 'REVEAL_RANK':
        rank_tag = info
        color_tag = None
    elif act_type is 'REVEAL_COLOR':
        color_tag = info
        rank_tag = None
    else:
        color_tag = None
        rank_tag = None
    
    action = {'action_type': act_type, 'target_offset': target_offset, 'rank': rank_tag, 'color': color_tag,
             'card_index': card_index}
    return action

# creating all possible actions
# (Note: currently it is not encoded which specific card is hinted at, only the player the hint goes to)
possible_actions = []

# create all play actions
for position in possible_actions_lexicon['card_index']:
    possible_actions.append(create_action_instance(act_type='PLAY', info=None, 
                                                        target_offset=None, card_index=position))

# create all discard actions
for position in possible_actions_lexicon['card_index']:
    possible_actions.append(create_action_instance(act_type='DISCARD', info=None, 
                                                        target_offset=None, card_index=position))      
        
# create all color hints
for offset in possible_actions_lexicon['target_offset']:
        for color in possible_actions_lexicon['color']:
            possible_actions.append(create_action_instance(act_type='REVEAL_COLOR', info=color, 
                                                        target_offset=offset, card_index=None))
            
# create all rank hints
for offset in possible_actions_lexicon['target_offset']:
        for rank in possible_actions_lexicon['rank']:
            possible_actions.append(create_action_instance(act_type='REVEAL_RANK', info=rank, 
                                                        target_offset=offset, card_index=None))
            
print(possible_actions)

[{'action_type': 'PLAY', 'target_offset': None, 'rank': None, 'color': None, 'card_index': 1}, {'action_type': 'PLAY', 'target_offset': None, 'rank': None, 'color': None, 'card_index': 2}, {'action_type': 'PLAY', 'target_offset': None, 'rank': None, 'color': None, 'card_index': 3}, {'action_type': 'PLAY', 'target_offset': None, 'rank': None, 'color': None, 'card_index': 4}, {'action_type': 'PLAY', 'target_offset': None, 'rank': None, 'color': None, 'card_index': 5}, {'action_type': 'DISCARD', 'target_offset': None, 'rank': None, 'color': None, 'card_index': 1}, {'action_type': 'DISCARD', 'target_offset': None, 'rank': None, 'color': None, 'card_index': 2}, {'action_type': 'DISCARD', 'target_offset': None, 'rank': None, 'color': None, 'card_index': 3}, {'action_type': 'DISCARD', 'target_offset': None, 'rank': None, 'color': None, 'card_index': 4}, {'action_type': 'DISCARD', 'target_offset': None, 'rank': None, 'color': None, 'card_index': 5}, {'action_type': 'REVEAL_COLOR', 'target_offs

Knowledge structure as in Eger algorithm with knowledge[plyr_nr][card_nr][col_nr][rank_nr] repr. # realisation left

In [17]:
knowledge_structure =[[],[],[]]  # imagine a 3 player game
knowledge_structure[0] = [  # of player 0
    # 0-th card
    [
        # 5 different colors in order of B,G,R,W,Y
        [2,0,2,2,1],  # 5 ranks
        [0,0,2,2,1],  
        [2,0,2,2,1],
        [2,0,2,2,1],
        [2,0,2,2,1]   
    ],
    # 1-st card
    [
        [0,0,2,2,0],  # blue is either rank2 or rank3
        [2,0,2,1,1],  # green
        [2,0,0,0,0],  # red
        [2,0,2,2,1],  # white
        [2,0,0,0,0]   # yellow
    ]
]

# unrealistic but suppose every plyr have same card possibilities for simplicity
knowledge_structure[1] = copy.deepcopy(knowledge_structure[0])
knowledge_structure[2] = copy.deepcopy(knowledge_structure[0])


num_player = 3
num_card = 2

# Utility
as given by Saskia

In [6]:
def utility(intention, estimated_board, card):
    score = 0

    if intention == 'play':
        # in intention is play and card is playable, this results in one more card on the fireworks.
        # reward this.
        if CardPlayable(card, estimated_board['fireworks']):
            score += 10

        # if intention is play and card is not playable at the time
        else:
            # punish loosing a card from stack
            score -= 1

            # punish getting a bomb depending on the number of bombs
            if estimated_board['life_tokens'] == 3:
                score -= 1
            elif estimated_board['life_tokens'] == 2:
                score -= 3
            elif estimated_board['life_tokens'] == 1:
                score -= 25

            # if card would still have been relevant in the future, punish loosing it depending on
            # the remaining copies of this card in the deck
            if not CardUseless(card, estimated_board['fireworks']):
                if remaining_copies(card, estimated_board['discard_pile']) == 2:
                    score -= 1
                elif remaining_copies(card, estimated_board['discard_pile']) == 1:
                    score -= 2
                elif remaining_copies(card, estimated_board['discard_pile']) == 0:
                    score -= 5


    elif intention == 'discard':
        # punish loosing a card from stack
        score -= 1

        # reward gaining a hint token:
        score += 0.5

        # punish discarding a playable card
        if CardPlayable(card, estimated_board['fireworks']):
            score -= 5

        # if card is not playable right now but would have been relevant in the future, punish
        # discarding it depending on the number of remaining copies in the game
        elif not CardUseless(card, estimated_board['fireworks']):
            if remaining_copies(card, estimated_board['discard_pile']) == 2:
                score -= 1
            elif remaining_copies(card, estimated_board['discard_pile']) == 1:
                score -= 2
            elif remaining_copies(card, estimated_board['discard_pile']) == 0:
                score -= 5

        # do we want to reward this additionally? I think rewarding gaining a hint token should be
        # enough, so nothing happens here
        elif CardUseless(card, estimated_board['fireworks']):
            pass

    elif intention == 'keep':
        # keeping a playable card is punished, because it does not help the game
        if CardPlayable(card, estimated_board['fireworks']):
            score -= 2

        # if card is not playable right now but is relevant in the future of the game reward keeping
        # this card depending on the remaining copies in the game
        elif not CardUseless(card, estimated_board['fireworks']):
            if remaining_copies(card, estimated_board['discard_pile']) == 2:
                score += 1
            elif remaining_copies(card, estimated_board['discard_pile']) == 1:
                score += 2
            elif remaining_copies(card, estimated_board['discard_pile']) == 0:
                score += 5

        # punish keeping a useless card
        elif CardUseless(card, estimated_board['fireworks']):
            score -= 1

    return score



intention = 'play'
# remember, that rank 0 in cards means rank 1, so rank = 0 and fireworks = 0 means card is playable!
card = {'color': 'B', 'rank': 2}

print('play: ', utility('play', estimated_board, card))
print('discard: ', utility('discard', estimated_board, card))
print('keep', utility('keep', estimated_board, card))



play:  10
discard:  -5.5
keep -2


## Speaker
$P(action|intention,context)$

In [7]:
# Bianca:
# Done: having defined theoretically possible actions (see above)
# TO DO: come up with legal actions for the Pragmatic Speaker from the Pragmatic Listener's point of view

In [8]:
def pragmatic_speaker(intention,context,card_index):
    alpha = 1.  # optimality
    numerator = np.zeros(len(possible_actions))
    for i in range(len(possible_actions)):
        action = possible_actions[i]
        numerator[i] = alpha * np.exp(emul_marg(action, intention, context, card_index))
    for act in possible_actions: 
        denom += alpha * np.exp(emul_marg(act, intention, context, card_index))
    return numerator / denom  # P(a|i,c)

### Emulation
should happen in the perspective of public agent

Sample multiple game board while keeping congruency with given action and avg. over them

e.g.) action: hint blue, game board positively identifies blue card and other negatively

### marginalise
P(i|a,c) = sum_b P(i,b|a,c) = sum_b P(i|a,c,b) * P(b|a,c)

p(b|a,c): how likely to get this specific card given the current knowledge structure

In [25]:
def emul_marg(action, intention, context, card_index):
#     knowledge_structure = context['estimated_hands']
#     TODO: store information such that retrieval is easy (something like list structure in Eger would be good)
#     also action must somehow influence possible interpretations of each realisation (a,c) -> b
#     summe = 0
#     for real in knowledge_structure:
#         summe += real_prob * utility(intention, real, card_index)  
        
    mass = 0
    threshold = 420
    # until enough prob. mass is sampled 
    while mass < threshold:  
        # only samples with prob > 0 is a valid sample
        while True:
            prob = 1
            # color, rank indicies for every card,plyr
            ci, ri = np.random.randint(low=0, high=5, size=(2,num_player,num_card)) 
            for pi in range(num_player):
                for i in range(num_card):
                    # independence for each card of each player
                    prob *= knowledge_structure[pi][i][ci[pi,i]][ri[pi,i]]
            if prob != 0:
                break
        
        # TODO: do we rly enumerate over all card of all player or only one player?
        realisation = copy.deepcopy(estimated_board)
        realisation['estimated_hands'] = [{'color': ci[pi,0], 'rank': ri[pi,0]},
                                          {'color': ci[pi,1], 'rank': ri[pi,1]},
                                         ]
        print('realisation', realisation)
        mass += prob * utility(intention, realisation, card_index)
    
    return summe  # U(a,i,c)

# Listener
$P(intention|action,context)$ for a single card

In [10]:
def int_prior(context):
    # TODO: return an intention sample depending on P(intention|context)
    # currently just use uniform prior regardless of context
    sample = pyro.sample("utt", dist.Categorical(probs=torch.ones(3) / 3))
    if sample == 0:
        return 'play'
    elif sample ==1:
        return 'keep'
    else:
        return 'discard'

In [11]:
def pragmatic_listener(action, context, card_index):
    def sample_pl(context, card_index):
        intention = int_prior(context) # P(i|c)
        pyro.sample("pragmatic_speaker", # P(a|i,c)
                    pragmatic_speaker(intention, context, card_index), obs=action) 
        return intention
    return Infer(sample_pl(context,card_index))

In [61]:
pragmatic_listener({'action_type': 'REVEAL_COLOR', 'target_offset': 1, 'rank': None, 'color': 'B', 'card_index': None},
                  estimated_board,
                  0)

NameError: name 'pragmatic_listener' is not defined

In [ ]:
knowledge_structure[1]